# Custom Tools

## 1. Way 1
** Using @tool decorator


This @tool decorator is the simplest way to define a custom tool. The decorator uses the function name as the tool name by default, but this can be overridden by passing a string as the first argument. Additionally, the decorator will use the function's docstring as the tool's description - so a docstring MUST be provided.

In [1]:
from langchain.tools import tool ,BaseTool
from langchain.pydantic_v1 import BaseModel, Field

**write a very simple function**

In [2]:
# write a very simple function
#decorate it with @tool
#place the docstring
@tool
def fnc_multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [3]:
type(fnc_multiply)


langchain_core.tools.StructuredTool

In [4]:
print(fnc_multiply.name)
print(fnc_multiply.description)
print(fnc_multiply.args)

fnc_multiply
fnc_multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


similar approach but this time we give it the tool a **separate name**

In [31]:
#similar approach but this time we give it the tool a **separate name**
@tool("multiply")
def fnc_multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

In [6]:
print(fnc_multiply.name)
print(fnc_multiply.description)
print(fnc_multiply.args)

fnc_multiply
fnc_multiply(a: int, b: int) -> int - Multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


## 2. Way 2
** Using the Tool class - split the function & tool

In [7]:
from langchain.tools import Tool

def fnc_multiply(a: int, b: int) -> int:
    """Multiply 'a' times 'b'."""
    return a * b

multiply:Tool = Tool(
    name="multiply",
    func=fnc_multiply,
    description="Use this tool to Multiply 'a' times 'b'."
                )

In [8]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
Use this tool to Multiply 'a' times 'b'.
{'tool_input': {'type': 'string'}}


## 3. Way 3
Subclass BaseTool

You can also explicitly define a custom tool by subclassing the BaseTool class. This provides maximal control over the tool definition, but is a bit more work.

In [9]:

from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Union


In [10]:
# a simple example !!

class Multiply(BaseTool):
    name="multiply"
    description="Use this tool to multiply two numbers"
    
    def _run(self,x:Union[int,float],y:Union[int,float])->float:
        return float(x * y)
    
    def _arun(self,x:Union[int,float],y:Union[int,float])->float:
        raise NotImplementedError("This tool does not support async")
    

In [13]:
# now more control !!
from langchain.pydantic_v1 import BaseModel, Field
from langchain.tools import BaseTool, StructuredTool, tool
from typing import Union

from typing import Optional, Type
from langchain.callbacks.manager import (
    AsyncCallbackManagerForToolRun,
    CallbackManagerForToolRun,
)

class MultiplyInput(BaseModel):
    x: int = Field(description="first number")
    y: int = Field(description="second number")

class Multiply(BaseTool):
    name="multiply"
    description="Use this tool to multiply two numbers"
    
    args_schema: Type[BaseModel] = MultiplyInput
    return_direct: bool = True
    
    def _run(self,x:Union[int,float],y:Union[int,float],run_manager:Optional[CallbackManagerForToolRun]=None)->float:
        return float(x * y)
    
    def _arun(self,x:Union[int,float],y:Union[int,float],run_manager:Optional[CallbackManagerForToolRun]=None)->float:
        """Use the tool asynchronously."""
        raise NotImplementedError("Calculator does not support async")


In [14]:
multiply = Multiply()
print(multiply.name)
print(multiply.description)
print(multiply.args)
print(multiply.return_direct)

multiply
Use this tool to multiply two numbers
{'x': {'title': 'X', 'description': 'first number', 'type': 'integer'}, 'y': {'title': 'Y', 'description': 'second number', 'type': 'integer'}}
True


## 4. Way 4
StructuredTool dataclass

You can also use a StructuredTool dataclass. This methods is a mix between the previous two. It's more convenient than inheriting from the BaseTool class, but provides more functionality than just using a decorator.

In [15]:
from langchain.tools import BaseTool, StructuredTool, tool

In [16]:
def fnc_multiply(a: int, b: int) -> int:
    """Multiply two numbers."""
    return a * b

multiply = StructuredTool.from_function(
    name="multiply",
    func=fnc_multiply,
    description="Use this method to multiply two numbers."
)

In [17]:
print(multiply.name)
print(multiply.description)
print(multiply.args)

multiply
multiply(a: int, b: int) -> int - Use this method to multiply two numbers.
{'a': {'title': 'A', 'type': 'integer'}, 'b': {'title': 'B', 'type': 'integer'}}


In [24]:
from HelperAIKey import fnc_getOpenAIKey , fnc_getTavilyKey
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(api_key=fnc_getOpenAIKey(), verbose=True)  # Put your openAI key here 

In [25]:
llm.invoke('multiply 2 and 3')

AIMessage(content='2 x 3 = 6', response_metadata={'token_usage': {'completion_tokens': 7, 'prompt_tokens': 13, 'total_tokens': 20}, 'model_name': 'gpt-3.5-turbo', 'system_fingerprint': None, 'finish_reason': 'stop', 'logprobs': None}, id='run-f043ef7d-989e-46c6-896e-32a4521d89ae-0')